In [1]:
from keras.models import load_model

model = load_model("./trained/cnn-sanbaka-norm.keras")

In [ ]:
talent_list = [
    {
        "name": "アンジュ・カトリーナ",
        "color": "#C83C35",
        "class_name": "ange"
    },
    {
        "name": "リゼ・ヘルエスタ",
        "color": "#42FFFF",
        "class_name": "lize",
    },
    {
        "name": "戌亥とこ",
        "color": "#92F3A4",
        "class_name": "toko"
    }
]

In [12]:
import pandas as pd

audio_filepath = "../data_collection/test-voice/sanbaka-sightseeing.mp4"
diarization = pd.read_csv("../pyannote/result-sightseeing-3.1.csv")

In [13]:
import librosa

# Parameters for mel-spectrogram
sample_rate = 22050  # Standard sample rate for Librosa. Basically equal to sample rate of the audio.
n_mels = 128         # Number of mel bands
n_fft = 2048         # Length of the FFT window -> ブログのやつでは173(分解能を11.5ms にするため！)
hop_length = 256     # Number of samples between successive frames
win_length = n_fft     # window length
segment_duration = 2  # seconds

y, sr = librosa.load(audio_filepath, sr = sample_rate)

/var/folders/n9/f02ch2416_7cp5wybnwwhh7r0000gn/T/ipykernel_6464/4084193733.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_filepath, sr = sample_rate)
/opt/miniconda3/envs/general/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
import numpy as np
import librosa
from math import ceil

labels = [d["name"] for d in talent_list]
colors = [d["color"] for d in talent_list]
class_names = [d["class_name"] for d in talent_list]

# Example entries
vtt_elements = []

predicted = []
readable_text = []

for index, row in diarization.iterrows():
    b = eval(row["text"])
    text = b.decode('utf-8')
    if text == "":
        predicted.append("")
        readable_text.append("")
        continue

    print(f"{row["start"]} - {row["end"]}: {row["speaker"]} said, ''{text}''")
    start = max(int(sr * row["start"]), 0)
    end = min(int(sr * row["end"]), len(y))
    turn = y[start:end]

    turn_len = len(turn)
    # 0-2, 1-3, 2-4,,,
    num_segments = int( turn_len / sr ) - segment_duration + 1
    matrix_list = []

    minimum_length = segment_duration * sr
    print(minimum_length, len(turn))
    if len(turn) < minimum_length:
        rep = ceil(minimum_length / len(turn))
        repeated = np.tile(turn, rep)
        print(repeated.shape, turn.shape)

        mel_spec = librosa.feature.melspectrogram(
            y=repeated[0:minimum_length], 
            sr=sr, n_fft=n_fft, win_length=win_length, hop_length=hop_length)

        ## dB単位に変換
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        # mel_spec_db_normalized = librosa.util.normalize(mel_spec_db)
        matrix_list.append(mel_spec_db)
    
    else:
        for segment_index in range(num_segments):
            segment = y[segment_index * sr : (segment_index + segment_duration) * sr ]

            mel_spec = librosa.feature.melspectrogram(
                y=segment, 
                sr=sr, n_fft=n_fft, win_length=win_length, hop_length=hop_length)

            ## dB単位に変換
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
            # mel_spec_db_normalized = librosa.util.normalize(mel_spec_db)
            matrix_list.append(mel_spec_db)

    X = np.empty((0, 128, 173))
    x_data = np.stack([m for m in matrix_list], axis=0)
    print(x_data.shape)
    X = np.concatenate((X, x_data), axis = 0)
    print(X.shape)
    X = X[..., np.newaxis]
    predictions = model.predict(X)
    predicted_indexs = np.argmax(predictions, axis=1)

    predicted_labels = [labels[i] for i in predicted_indexs]

    values, counts = np.unique(predicted_labels, return_counts=True)
    print(values)
    print(counts)

    print(f"Most Dominant: {values[np.argmax(counts)]}")

    color = colors[np.argmax(counts)]
    class_name = class_names[np.argmax(counts)]

    vtt_elements.append({
        'start': row["start"],
        'end': row["end"],
        'text': text,
        'color': color,
        'class_name': class_name
    })

    predicted.append(values[np.argmax(counts)])
    readable_text.append(text)

diarization["text"] = readable_text
diarization["predicted"] = predicted
diarization.to_csv("../pyannote/result-sightseeing-3.1.predicted.csv", index=False)

0.03096875 - 6.24096875: b'SPEAKER_01' said, ''音量調整したけど、視聴者さん側にも念のため聞いてもらおうかしましょう了解''
44100 136931
(5, 128, 173)
(5, 128, 173)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
['アンジュ・カトリーナ' '戌亥とこ']
[3 2]
Most Dominant: アンジュ・カトリーナ
8.215343750000002 - 8.704718750000001: b'SPEAKER_01' said, ''だれ?''
44100 10791
(53955,) (10791,)
(1, 128, 173)
(1, 128, 173)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
['戌亥とこ']
[1]
Most Dominant: 戌亥とこ
9.97034375 - 10.476593750000005: b'SPEAKER_01' said, ''あ、だ…''
44100 11162
(44648,) (11162,)
(1, 128, 173)
(1, 128, 173)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
['アンジュ・カトリーナ']
[1]
Most Dominant: アンジュ・カトリーナ
10.62846875 - 11.84346875: b'SPEAKER_01' said, ''お待たせ!まった!''
44100 26791
(53582,) (26791,)
(1, 128, 173)
(1, 128, 173)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
['戌亥とこ']
[1]
Most Dominant: 戌亥とこ
12.04596875 - 15.15096875: b'SPEAKER_01' said, ''2次参事所属公式美少女連輪試合場の''
44100 68465
(2, 128, 173)
(2, 128, 173)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
['アンジュ・カトリーナ']
[2]
Most Dominant: アンジュ・カトリーナ

In [ ]:
def format_time(secs: float) -> str:
    """Convert seconds as float to SRT time format (hh:mm:ss,ms)."""
    hours =  int(secs) // 3600
    minutes = (int(secs) % 3600) // 60
    seconds = ( int(secs) % 60 ) // 1
    milliseconds = int((secs - (int(secs) // 1)) * 1000 )
    return f"{str(hours).zfill(2)}:{str(minutes).zfill(2)}:{str(seconds).zfill(2)}.{str(milliseconds).zfill(3)}"

def create_srt_entry(index: int, start_time: float, end_time: float, text: str, class_name: str) -> str:
    """Create a formatted SRT entry."""
    start_time_str = format_time(start_time)
    end_time_str = format_time(end_time)
    formatted_text = f"<c.{class_name}>{text}</c>"
    return f"{index}\n{start_time_str} --> {end_time_str}\n{formatted_text}\n"

def write_srt_file(entries: list, output_file: str):
    """Write entries to an SRT file."""
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write("WEBVTT\n\n")
        for i, entry in enumerate(entries, start=1):
            file.write(create_srt_entry(i, entry['start'], entry['end'], entry['text'], entry['class_name']))
            file.write("\n")

# Write to an SRT file
write_srt_file(vtt_elements, 'sanbaka-sightseeing-cnn-norm.vtt')
